In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.stats import norm
from statsmodels.regression.linear_model import OLS

%matplotlib inline

In [2]:
df = pd.read_csv('USCrime.csv', sep=';')

In [3]:
df['(Intercept)'] = 1

In [4]:
col_mapping = {
    "R" : "CrimeRate",
    "Age": "Age",
    "S": "SouthernState",
    "Ed": "Education",
    "Ex0": "Expanditure1960",
    "Ex1": "Expanditure1959",
    "LF": "Labor",
    "M": "NumberOfMales",
    "N": "Population",
    "NW": "NonWhite",
    "U1": "Unemployment14-24",
    "U2": "Unemployment25-39",
    "W": "Wealth"
}

df.rename(columns=col_mapping, inplace=True)
df.head()

,CrimeRate,Age,SouthernState,Education,Expanditure1960,Expanditure1959,Labor,NumberOfMales,Population,NonWhite,Unemployment14-24,Unemployment25-39,Wealth,X,(Intercept)
0,79.1,151,1,91,58,56,510,950,33,301,108,41,394,261,1
1,163.5,143,0,113,103,95,583,1012,13,102,96,36,557,194,1
2,57.8,142,1,89,45,44,533,969,18,219,94,33,318,250,1
3,196.9,136,0,121,149,141,577,994,157,80,102,39,673,167,1
4,123.4,141,0,121,109,101,591,985,18,30,91,20,578,174,1


## MULTIPLE REGRESSION (EXAMPLE 13.14)

In [5]:
features = [
 '(Intercept)',
 'Age',
 'SouthernState',
 'Education',
 'Expanditure1960',
 'Labor',
 'NumberOfMales',
 'Population',
 'Unemployment14-24',
 'Unemployment25-39',
 'Wealth'
]

Y = df["CrimeRate"]
X = df[features]

In [6]:
# BETAS
inverse = np.linalg.inv(X.T.dot(X))
betas = inverse.dot(X.T).dot(Y)

# STANDARD ERROR
eps = X.dot(betas) - Y
sigma_squared = ((eps * eps).sum())/(len(X) - len(X.columns))
standard_errors = np.array([math.sqrt((sigma_squared * inverse)[i][i]) for i in range(len(X.columns))])

# T VALUE
t_value = betas / standard_errors

# P VALUE
p_value = np.vectorize(lambda x : 2*norm.cdf(-1 * abs(x)))(t_value)

# CREATE RESULT
result = pd.DataFrame([betas, standard_errors, t_value, p_value],
                      columns=features, 
                      index=['Beta(j)', 'standard_errors', 't_value', 'p_value']).T
result["Beta(j)"] = np.round(result["Beta(j)"], 2)
result["standard_errors"] = np.round(result["standard_errors"], 2)
result["t_value"] = np.round(result["t_value"], 2)
result["p_value"] = np.round(result["p_value"], 3)

result

,Beta(j),standard_errors,t_value,p_value
(Intercept),-589.40,167.59,-3.52,0.000
Age,1.04,0.45,2.33,0.020
SouthernState,11.29,13.25,0.85,0.394
Education,1.18,0.68,1.73,0.084
Expanditure1960,0.96,0.25,3.86,0.000
Labor,0.11,0.15,0.69,0.489
NumberOfMales,0.30,0.22,1.36,0.173
Population,0.09,0.14,0.65,0.514
Unemployment14-24,-0.68,0.48,-1.42,0.156
Unemployment25-39,2.15,0.95,2.26,0.024


## MODEL SELECTION (EXAMPLE 13.6)

In [8]:
X = df[features]
Y = df["CrimeRate"]
regr = OLS(Y, X).fit()
result = pd.DataFrame(regr.aic, index=["AIC"], columns=features)

while True:
    previous = result
    datas = dict()
    for feature in X.columns:
        newX = X.drop(columns=feature)
        regr = OLS(Y, newX).fit()
        datas[feature] = regr.aic
    result = pd.DataFrame(datas, index=["AIC"])
    difference = abs((previous[result.columns] - result).loc["AIC"].sum())
    if difference < 1:
        print("Breaking with difference {} ... ".format(difference))
        print("Kept features : {}".format(", ".join(result.columns)))
        break
    print("Dropping {} ...".format(result.T.idxmin()['AIC']))
    X = X.drop(columns=result.T.idxmin()['AIC'])

Dropping Population ...
Dropping SouthernState ...
Dropping Labor ...
Dropping Wealth ...
Breaking with difference 0.19667801383667438 ... 
Kept features : (Intercept), Age, Education, Expanditure1960, NumberOfMales, Unemployment14-24, Unemployment25-39


In [9]:
# BETAS
inverse = np.linalg.inv(X.T.dot(X))
betas = np.round(inverse.dot(X.T).dot(Y), 2)

pd.DataFrame(betas, index=list(X.columns), columns=["Betas"])

,Betas
(Intercept),-565.12
Age,1.24
Education,0.75
Expanditure1960,0.87
NumberOfMales,0.34
Unemployment14-24,-0.86
Unemployment25-39,2.31
